In [27]:
import numpy as np
import scipy.constants as scp
import itertools
import os
import re

In [28]:
eV_to_J, _, _ = scp.physical_constants['electron volt-joule relationship']
MeV_to_eV = 1.0e6
J_to_erg = 1.0e7
MeV_to_erg = MeV_to_eV * eV_to_J * J_to_erg

In [29]:
files = ['A21_OFNeNaMg_ScrExp.odat',
         'A20_OFNeNaMg_ScrExp.odat',
         'A19_OFNeNa_ScrExp.odat',
         'A18_OFNe_ScrExp.odat',
         'A17_FO_ScrExp.odat',
         'A22_FNeNaMg_ScrExp.odat',
         'A23_FNeNaMgAl_ScrExp.odat',
         'A24_NeNaMgAlSi_ScrExp.odat',
         'A25_NeNaMgAlSi_ScrExp.odat',
         'A26_NaMgAlSi_ScrExp.odat',
         'A27_NaMgAlSiP_ScrExp.odat',
         'A28_NaMgAlSiPS_ScrExp.odat']

In [30]:
def weak_decay_read(file):

    headers = []
    data = []
    desc = []

    with open(file) as f:
        _ = f.readline()
        while (l := f.readline()):
            if l == "\n":
                continue
            headers.append([])
            if l.startswith('!'):
                headers[-1].append(l.rstrip('\n'))
            
            while (l := f.readline().rstrip('\n')):
                if l.startswith('!'):
                    headers[-1].append(l)

            # sections are separated by empty lines
            data.append(np.genfromtxt(itertools.takewhile(lambda x: x.rstrip('\n'), f), autostrip=True))

    if not headers[-1]:
        headers.pop(-1)
        data.pop(-1)

    for head, chunk in zip(headers, data):

        for line in head:
            if line.startswith('!'):
                #print(line)
                if re.search('e-capture', line):
                    out_str = 'e-capture'  
                elif re.search('beta-decay', line):
                    out_str = 'beta-decay'
                else:
                    out_str = None

                elements = re.findall('[0-9]{1,2}[A-Za-z]{1,2}', line) 
                el1 = elements[0].lower()
                el2 = elements[1].lower()
                break
            else:
                continue

        if out_str=='e-capture':
            name=f"{el1}-{el2}_electroncapture.dat"
        elif out_str=='beta-decay':
            name=f"{el1}-{el2}_betadecay.dat"
        else:
            name=None

        if (out_str=='e-capture' or out_str=='beta-decay'):
            head[-2] = '!Log(rhoY)        Log(T)            mu           dQ           Vs           Log(e-cap-rate)   nu-energy-loss   gamma-energy'
            head[-1] = '!Log(g/cm^3)      Log(K)            erg          erg          erg          Log(1/s)          Log(erg/s)       Log(erg/s)'
            desc.append((name, head, chunk))

    return desc

In [31]:
def weak_decay_write(desc):

    for output_file in desc:
        
        name, head, chunk = output_file
        with open(name, "w") as output:

                rho, temp, mu, dq, vs, rate, nu_energy_loss, gamma_energy = chunk.T
                mu *= MeV_to_erg 
                dq *= MeV_to_erg 
                vs *= MeV_to_erg 

                rate = rate
                nu_energy_loss = nu_energy_loss + np.log10(MeV_to_erg)
                gamma_energy = gamma_energy + np.log10(MeV_to_erg)

                for line in head:
                    output.write(line)
                    output.write("\n")

                for i in range(len(rho)):
                    output.write(f"{rho[i]:>.2f} {temp[i]:>17.2f} {mu[i]:>24.5e} {dq[i]:>12.5e} {vs[i]:>12.5e} {rate[i]:>13.5e}  {nu_energy_loss[i]:16.5e} {gamma_energy[i]:16.5e}\n")


In [32]:
for file in files:
    desc = weak_decay_read(file)
    weak_decay_write(desc)

In [33]:
for file in files:
    try:
        desc = weak_decay_read(file)
    except:
        print(file)